# Step 1. JSON 데이터 업로드 (Full-Text Search)
OpenSearch에 JSON 데이터를 Nori(한국어) 분석기와 함께 업로드합니다.

## 배경 지식

### BM25 (Best Matching 25)
OpenSearch의 **기본 검색 알고리즘**으로, 문서와 질의 간의 관련성을 점수로 계산합니다.
- **단어 빈도 (TF)**: 문서에 단어가 많이 등장할수록 더 중요
- **역문서 빈도 (IDF)**: 전체 문서에서 드물게 등장하는 단어일수록 더 중요
- **문서 길이 정규화**: 너무 긴 문서가 높은 점수를 얻는 걸 방지

### 역인덱스 (Inverted Index)
- **term**: 문서에 등장하는 개별 단어/토큰 (형태소 단위)
- **posting list**: 특정 term이 등장하는 모든 문서의 리스트 (위치 정보 포함)
- 단어를 key로, 해당 단어가 포함된 문서 ID 리스트를 값으로 가지는 매핑

### Nori 분석기
OpenSearch 공식 **한국어 형태소 분석기**입니다. 한국어 텍스트를 의미 있는 토큰으로 분리하여 검색 품질을 높입니다.

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0

## 1. 설정 (Configuration)
OpenSearch 접속 정보를 입력합니다. `HOST` 값을 본인의 환경에 맞게 수정해주세요.

In [ ]:
import os, json

# Step 0에서 저장한 설정 불러오기
try:
    with open("../config.json") as f:
        _config = json.load(f)
    print("✅ config.json 로드 완료")
except FileNotFoundError:
    raise FileNotFoundError("❌ config.json을 찾을 수 없습니다. Step 0 노트북을 먼저 실행해주세요.")

# OpenSearch Serverless 엔드포인트
HOST = _config.get("OPENSEARCH_HOST")
if not HOST:
    raise ValueError("❌ config.json에 OPENSEARCH_HOST 값이 없습니다. Step 0 노트북을 먼저 실행해주세요.")
DEFAULT_REGION = _config.get("DEFAULT_REGION", "ap-northeast-2")
PROFILE = _config.get("PROFILE", "skku-opensearch-session")

## 2. OpenSearch 클라이언트 생성

In [3]:
import boto3
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection

service = 'aoss'
credentials = boto3.Session(profile_name=PROFILE).get_credentials()
auth = AWSV4SignerAuth(credentials, DEFAULT_REGION, service)

client = OpenSearch(
    hosts=[{'host': HOST, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

print("OpenSearch 클라이언트 생성 완료")

OpenSearch 클라이언트 생성 완료


## 3. 인덱스 설정 및 생성
Nori 분석기를 적용한 인덱스 매핑을 정의하고 인덱스를 생성합니다.

In [4]:
from time import sleep

index_name = 'json-test'

# nori 분석기를 적용한 인덱스 매핑 및 설정 정의
index_body = {
    "settings": {
        "index": {
            "analysis": {
                "analyzer": {
                    "korean_nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer",
                        "filter": [
                            "nori_part_of_speech",
                            "nori_readingform",
                            "lowercase"
                        ]
                    }
                }
            },
            "number_of_shards": 1,
            "number_of_replicas": 1
        }
    },
    "mappings": {
        "properties": {
            "title": {
                "type": "text",
                "analyzer": "korean_nori_analyzer"
            },
            "content": {
                "type": "text",
                "analyzer": "korean_nori_analyzer"
            },
            "author": {"type": "keyword"},
            "timestamp": {"type": "date"}
        }
    }
}

if not client.indices.exists(index=index_name):
    print(f"Creating new index '{index_name}' with nori analyzer...")
    response = client.indices.create(index=index_name, body=index_body)
    print(f"Index '{index_name}' created successfully.")
    sleep(5)
else:
    print(f"Index '{index_name}' already exists.")

Creating new index 'json-test' with nori analyzer...
Index 'json-test' created successfully.


## 4. JSON 데이터 로드 및 업로드
Bulk API를 사용하여 JSON 데이터를 대량으로 업로드합니다.

In [12]:
import json
from opensearchpy import helpers

json_file_path = '../../data/json_data.json'

with open(json_file_path, 'r', encoding='utf-8') as f:
    documents = json.load(f)
print(f"Successfully loaded {len(documents)} documents from '{json_file_path}'.")

def generate_bulk_actions(docs, idx_name):
    """Bulk API에 맞는 형식으로 데이터를 가공하는 함수"""
    for doc in docs:
        yield {
            "_index": idx_name,
            "_source": doc
        }

print("Starting data upload...")
success, failed = helpers.bulk(client, generate_bulk_actions(documents, index_name))

print(f"Successfully indexed {success} documents.")
if failed:
    print(f"Failed to index {len(failed)} documents.")
    for i, item in enumerate(failed):
        if i >= 5: break
        print(f"Failed item {i+1}: {item}")

Successfully loaded 50 documents from '../../data/json_data.json'.
Starting data upload...
Successfully indexed 50 documents.


## 5. 업로드된 데이터 검색해보기
데이터가 잘 업로드되었는지 Python 코드로 검색해봅니다.

In [14]:
# 전체 검색 (match_all)
response = client.search(
    index=index_name,
    body={"query": {"match_all": {}}}
)
print(f"전체 문서 수: {response['hits']['total']['value']}")

# 자연어 검색 (match) - Nori 분석기가 적용된 title 필드로 검색
response = client.search(
    index=index_name,
    body={"query": {"match": {"title": "오픈서치"}}}
)
print(f"\n--- '오픈서치' 검색 결과 ({response['hits']['total']['value']}건) ---")
for hit in response['hits']['hits']:
    print(f"  [{hit['_score']:.4f}] {hit['_source']['title']}")

전체 문서 수: 50

--- '오픈서치' 검색 결과 (1건) ---
  [2.6788] Meta, Llama 3 오픈소스 AI 모델 공개


## 실습 과제

### 과제 1: 다양한 검색 쿼리 실행해보기
아래 셀들에서 OpenSearch의 다양한 검색 기능을 체험해보세요.

> 💡 같은 쿼리를 OpenSearch Dashboard의 **Dev Tools**에서도 실행해볼 수 있습니다.
> 참고 파일: `devtools/index.query`, `devtools/json_data_search.query`, `devtools/upload_data.query`

### 과제 2: 검색어 변경해보기
각 셀의 `# TODO:` 주석이 있는 부분의 값을 바꿔가며 결과를 비교해보세요.

#### 인덱스 매핑 확인
인덱스의 필드 구조(매핑)를 확인합니다. Dashboard Dev Tools에서 `GET json-test/_mapping`과 동일합니다.

In [ ]:
# 인덱스 매핑 확인
mapping = client.indices.get_mapping(index=index_name)
print(json.dumps(mapping, indent=2, ensure_ascii=False))

#### 전체 검색 (match_all)
인덱스의 모든 문서를 조회합니다.

In [ ]:
# 전체 검색
response = client.search(
    index=index_name,
    body={"query": {"match_all": {}}}
)
print(f"전체 문서 수: {response['hits']['total']['value']}건")
for i, hit in enumerate(response['hits']['hits'][:5]):
    print(f"  [{i+1}] {hit['_source'].get('title', 'N/A')}")

#### 자연어 검색 (match)
Nori 분석기가 한국어를 형태소 단위로 분리하여 검색합니다.

In [ ]:
# TODO: 검색어를 바꿔보세요!
search_field = "title"
search_term = "오픈서치"

response = client.search(
    index=index_name,
    body={"query": {"match": {search_field: search_term}}}
)
print(f"--- '{search_term}' 검색 결과 ({response['hits']['total']['value']}건) ---")
for i, hit in enumerate(response['hits']['hits']):
    print(f"  [{hit['_score']:.4f}] {hit['_source'].get('title', 'N/A')}")

#### keyword 검색 (term)
`keyword` 타입 필드는 정확히 일치하는 값만 검색됩니다. `text` 타입과 달리 형태소 분석을 하지 않습니다.

In [ ]:
# TODO: 검색할 작성자를 바꿔보세요!
search_author = "김개발"

response = client.search(
    index=index_name,
    body={"query": {"term": {"author": search_author}}}
)
print(f"--- author='{search_author}' 검색 결과 ({response['hits']['total']['value']}건) ---")
for i, hit in enumerate(response['hits']['hits']):
    print(f"  [{i+1}] {hit['_source'].get('title', 'N/A')} (작성자: {hit['_source'].get('author', 'N/A')})")

#### 단일 문서 추가 및 검색
Bulk API 외에 단일 문서를 직접 추가할 수도 있습니다.

In [ ]:
from time import sleep

# 단일 문서 추가
new_doc = {
    "post_id": 51,
    "title": "새로운 데이터 추가 테스트",
    "content": "Dev Tools를 사용하여 단일 문서를 색인합니다. 한국어 분석이 잘 되는지 확인합니다.",
    "author": "test-user",
    "timestamp": "2025-01-01T10:00:00"
}

response = client.index(index=index_name, body=new_doc)
print(f"✅ 문서 추가 완료 (id: {response['_id']})")

sleep(1)  # 인덱싱 대기

# 추가한 문서 검색
response = client.search(
    index=index_name,
    body={"query": {"match": {"title": "데이터 추가 테스트"}}}
)
print(f"\n--- 추가한 문서 검색 결과 ({response['hits']['total']['value']}건) ---")
for hit in response['hits']['hits']:
    print(f"  제목: {hit['_source']['title']}")
    print(f"  내용: {hit['_source']['content']}")
    print(f"  작성자: {hit['_source']['author']}")

#### content 필드로 검색
제목(title)이 아닌 본문(content) 필드로 검색해봅니다.

In [ ]:
# TODO: 검색어를 바꿔서 실행해보세요!
search_keyword = "AWS"  # 원하는 검색어로 변경

response = client.search(
    index=index_name,
    body={"query": {"match": {"content": search_keyword}}}
)

print(f"--- '{search_keyword}' 검색 결과 ({response['hits']['total']['value']}건) ---")
for i, hit in enumerate(response['hits']['hits']):
    print(f"\n[{i+1}] score: {hit['_score']:.4f}")
    print(f"    제목: {hit['_source'].get('title', 'N/A')}")
    print(f"    내용: {hit['_source'].get('content', 'N/A')[:100]}...")